# MTH5 to StationXML and miniseed

In [1]:
from pathlib import Path
from mth5.mth5 import MTH5

In [2]:
h5_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl008.h5")

In [3]:
with MTH5() as m:
    m.open_mth5(h5_path)
    experiment = m.to_experiment()
    stream_list = []
    for row in m.run_summary.itertuples():
        if row.has_data():
            run_ts = m.get_run(row.run_hdf5_reference).to_run_ts()
            stream_list.append(run_ts.to_obspy_stream)

2025-03-06T10:15:25.843550-0800 | INFO | mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl008.h5


ValueError: The input run id is None. Input a string or integer.